# Importante Pacotes e Carregando o Dataset

In [1]:
#Importação de pacotes
import locale
import warnings
import pandas as pd

warnings.filterwarnings("ignore") 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
#Importação do arquivo de dados
df_original = pd.read_excel("dados_exemplos_v1.xlsx", dtype={'CEP_CLIENTE': 'str', 'CEP_VENDEDOR':'str'})

# Analise Exploratória - Visão Geral
<br>
Aqui o objetivo desta analise é ter uma visão geral do conjunto de dados, compreendendo seu tamanho, variaveis, 
tipo de dados e período dos dados coletados.

In [3]:
#Tamanho do conjunto de dados. 200 linhas e 5 variáveis
df_original.shape

(200, 5)

In [4]:
#Visão geral do conjunto de dados
df_original.head()

,CODIGO_CLIENTE,DATA_VENDA,CODIGO_VENDEDOR,CEP_CLIENTE,CEP_VENDEDOR
0,5695,11/09/2015,969022,78795000,74280380
1,5705,17/05/2016,969022,74905090,74280380
2,5711,22/09/2015,969022,78725160,74280380
3,5727,28/09/2015,969031,75904520,74535290
4,5994,31/05/2016,967231,43900000,40385640


In [5]:
# Vamos avaliar os tipos de variaveis
# Precisamos confirmar que a variacel CEP_CLIENTE e CEP_VENDEDOR esteja como String.
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CODIGO_CLIENTE   200 non-null    int64 
 1   DATA_VENDA       200 non-null    object
 2   CODIGO_VENDEDOR  200 non-null    int64 
 3   CEP_CLIENTE      200 non-null    object
 4   CEP_VENDEDOR     200 non-null    object
dtypes: int64(2), object(3)
memory usage: 7.9+ KB


In [8]:
# Vamos criar um DataFrame separado somente com os CEP´s e depois manter somente os CEP´s únicos.
# Dessa forma, consultaremos a API somente uma vez para cada CEP e posteriormente faremos um MERGE e unificaremos os dados.
df_ceps_clientes = df_original["CEP_CLIENTE"]
df_ceps_clientes = pd.DataFrame(df_ceps_clientes)
df_ceps_clientes.drop_duplicates(inplace = True)
df_ceps_clientes.head()

,CEP_CLIENTE
0,78795000
1,74905090
2,78725160
3,75904520
4,43900000


In [9]:
# Vamos criar um DataFrame separado somente com os CEP´s e depois manter somente os CEP´s únicos.
# Dessa forma, consultaremos a API somente uma vez para cada CEP e posteriormente faremos um MERGE e unificaremos os dados.
df_ceps_vendedor = df_original["CEP_VENDEDOR"]
df_ceps_vendedor = pd.DataFrame(df_ceps_vendedor)
df_ceps_vendedor.drop_duplicates(inplace = True)
df_ceps_vendedor.head()

,CEP_VENDEDOR
0,74280380
3,74535290
4,40385640
5,14080210
6,74883020


In [10]:
print("Ceps únicos de vendedores:", df_ceps_vendedor.count()) 
print("Ceps únicos de clientes:", df_ceps_clientes.count())

Ceps únicos de vendedores: CEP_VENDEDOR    17
dtype: int64
Ceps únicos de clientes: CEP_CLIENTE    167
dtype: int64


In [11]:
import urllib.request
import json

for idx, row in df_ceps_vendedor.iterrows():
        
        cep = row.CEP_VENDEDOR
        url = 'https://viacep.com.br/ws/%s/json/' % cep        
        headers = {'User-Agent': 'Autociencia/1.0'}
        requisicao = urllib.request.Request(url, headers=headers, method='GET')
        cliente = urllib.request.urlopen(requisicao)
        conteudo = cliente.read().decode('utf-8')
        endereco = json.loads(conteudo)        
        cliente.close()
        print(cep)
        
        if "cep" in endereco:
            df_ceps_vendedor.at[idx, 'LOGRADOURO_VENDEDOR'] = endereco['logradouro']
            df_ceps_vendedor.at[idx, 'BAIRRO_VENDEDOR'] = endereco['bairro']
            df_ceps_vendedor.at[idx, 'CIDADE_VENDEDOR'] = endereco['localidade']
            df_ceps_vendedor.at[idx, 'ESTADO_VENDEDOR'] = endereco['uf']
            
        else:       
            df_ceps_vendedor.at[idx, 'LOGRADOURO_VENDEDOR'] = 'Não Localizado'
            df_ceps_vendedor.at[idx, 'BAIRRO_VENDEDOR'] = 'Não Localizado'
            df_ceps_vendedor.at[idx, 'CIDADE_VENDEDOR'] = 'Não Localizado' 
            df_ceps_vendedor.at[idx, 'ESTADO_VENDEDOR'] = 'Não Localizado'

74280380
74535290
40385640
14080210
74883020
38401515
65070020
07097360
38408482
75800012
14095020
14407050
72310060
57081285
74610010
11350160
74363410


In [12]:
df_ceps_vendedor.head()

,CEP_VENDEDOR,LOGRADOURO_VENDEDOR,BAIRRO_VENDEDOR,CIDADE_VENDEDOR,ESTADO_VENDEDOR
0,74280380,Avenida T 15,Nova Suiça,Goiânia,GO
3,74535290,Rua Doutor Gil Lino,Setor Coimbra,Goiânia,GO
4,40385640,Rua José Tibério,Boa Vista de São Caetano,Salvador,BA
5,14080210,Rua Capitão Salomão,Campos Elíseos,Ribeirão Preto,SP
6,74883020,Rua Iraí,Alphaville Flamboyant Residencial Araguaia,Goiânia,GO


In [13]:
for idx, row in df_ceps_clientes.iterrows():
        
        cep = row.CEP_CLIENTE
        url = 'https://viacep.com.br/ws/%s/json/' % cep        
        headers = {'User-Agent': 'Autociencia/1.0'}
        requisicao = urllib.request.Request(url, headers=headers, method='GET')
        cliente = urllib.request.urlopen(requisicao)
        conteudo = cliente.read().decode('utf-8')
        endereco = json.loads(conteudo)        
        cliente.close()
        print(cep)
        
        if "cep" in endereco:
            df_ceps_clientes.at[idx, 'LOGRADOURO_CLIENTE'] = endereco['logradouro']
            df_ceps_clientes.at[idx, 'BAIRRO_CLIENTE'] = endereco['bairro']
            df_ceps_clientes.at[idx, 'CIDADE_CLIENTE'] = endereco['localidade']
            df_ceps_clientes.at[idx, 'ESTADO_CLIENTE'] = endereco['uf']
            
        else:       
            df_ceps_clientes.at[idx, 'LOGRADOURO_CLIENTE'] = 'Não Localizado'
            df_ceps_clientes.at[idx, 'BAIRRO_CLIENTE'] = 'Não Localizado'
            df_ceps_clientes.at[idx, 'CIDADE_CLIENTE'] = 'Não Localizado' 
            df_ceps_clientes.at[idx, 'ESTADO_CLIENTE'] = 'Não Localizado'

78795000
74905090
78725160
75904520
43900000
14061350
75370000
74491628
75392702
38410214
65110000
74711010
72410305
71570818
71805416
72241606
72583500
71926250
70680162
07091000
72255310
72220054
70680150
72622219
72870046
73150075
73252200
73751030
72876565
72007545
75135740
73807305
72155209
72130580
72250106
73813645
72308204
73825000
71261530
72312102
72225052
71805128
75860000
75904118
75828000
38425385
72425050
72235816
72225110
72306107
71727124
74675530
73752196
73813832
72870213
72236800
72145816
73105010
38405274
74130011
72260869
72313715
73340505
71805407
71805410
33350000
14850000
70802020
72341205
73950000
75830000
72210075
72803340
72225203
72145626
72927149
72921246
72914215
62785000
60355250
61760000
60191674
60745660
57601280
60310770
75383585
74810240
85555000
75390331
57160000
57500000
74565220
74365410
74770010
72016240
74934040
07083240
03366010
11730000
02078010
57100000
64023580
64045580
11335310
11349150
11347050
06852180
11700300
22250040
78735775
78730231
7

In [14]:
df_ceps_clientes.head()

,CEP_CLIENTE,LOGRADOURO_CLIENTE,BAIRRO_CLIENTE,CIDADE_CLIENTE,ESTADO_CLIENTE
0,78795000,,,Pedra Preta,MT
1,74905090,Rua Santa Helena,Jardim das Esmeraldas,Aparecida de Goiânia,GO
2,78725160,Rua Elza Oliveira,Jardim Primavera II,Rondonópolis,MT
3,75904520,Rua 7,Setor Dona Gercina,Rio Verde,GO
4,43900000,,,São Francisco do Conde,BA


In [15]:
df_ceps_vendedor["BAIRRO_CIDADE"] = df_ceps_vendedor["BAIRRO_VENDEDOR"] + "-" + df_ceps_vendedor["CIDADE_VENDEDOR"]
df_ceps_vendedor.head()

,CEP_VENDEDOR,LOGRADOURO_VENDEDOR,BAIRRO_VENDEDOR,CIDADE_VENDEDOR,ESTADO_VENDEDOR,BAIRRO_CIDADE
0,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,Nova Suiça-Goiânia
3,74535290,Rua Doutor Gil Lino,Setor Coimbra,Goiânia,GO,Setor Coimbra-Goiânia
4,40385640,Rua José Tibério,Boa Vista de São Caetano,Salvador,BA,Boa Vista de São Caetano-Salvador
5,14080210,Rua Capitão Salomão,Campos Elíseos,Ribeirão Preto,SP,Campos Elíseos-Ribeirão Preto
6,74883020,Rua Iraí,Alphaville Flamboyant Residencial Araguaia,Goiânia,GO,Alphaville Flamboyant Residencial Araguaia-Goi...


In [16]:
df_ceps_clientes["BAIRRO_CIDADE"] = df_ceps_clientes["BAIRRO_CLIENTE"] + "-" + df_ceps_clientes["CIDADE_CLIENTE"]
df_ceps_clientes.head()

,CEP_CLIENTE,LOGRADOURO_CLIENTE,BAIRRO_CLIENTE,CIDADE_CLIENTE,ESTADO_CLIENTE,BAIRRO_CIDADE
0,78795000,,,Pedra Preta,MT,-Pedra Preta
1,74905090,Rua Santa Helena,Jardim das Esmeraldas,Aparecida de Goiânia,GO,Jardim das Esmeraldas-Aparecida de Goiânia
2,78725160,Rua Elza Oliveira,Jardim Primavera II,Rondonópolis,MT,Jardim Primavera II-Rondonópolis
3,75904520,Rua 7,Setor Dona Gercina,Rio Verde,GO,Setor Dona Gercina-Rio Verde
4,43900000,,,São Francisco do Conde,BA,-São Francisco do Conde


In [17]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
#!pip install geopy
import geopy.distance 


locator = Nominatim(user_agent="localizacao")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df_ceps_vendedor['BAIRRO_CIDADE_GEOLOCALIZADO'] = df_ceps_vendedor['BAIRRO_CIDADE'].apply(geocode)
df_ceps_vendedor.head()

,CEP_VENDEDOR,LOGRADOURO_VENDEDOR,BAIRRO_VENDEDOR,CIDADE_VENDEDOR,ESTADO_VENDEDOR,BAIRRO_CIDADE,BAIRRO_CIDADE_GEOLOCALIZADO
0,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,Nova Suiça-Goiânia,"(Setor Nova Suiça, Goiânia, Região Geográfica ..."
3,74535290,Rua Doutor Gil Lino,Setor Coimbra,Goiânia,GO,Setor Coimbra-Goiânia,"(Setor Coimbra, Goiânia, Região Geográfica Ime..."
4,40385640,Rua José Tibério,Boa Vista de São Caetano,Salvador,BA,Boa Vista de São Caetano-Salvador,"(Boa Vista de São Caetano, Salvador, Região Ge..."
5,14080210,Rua Capitão Salomão,Campos Elíseos,Ribeirão Preto,SP,Campos Elíseos-Ribeirão Preto,"(Campos Elíseos, Ribeirão Preto, Região Imedia..."
6,74883020,Rua Iraí,Alphaville Flamboyant Residencial Araguaia,Goiânia,GO,Alphaville Flamboyant Residencial Araguaia-Goi...,"(Alphaville Flamboyant Residencial Araguaia, G..."


In [18]:
locator = Nominatim(user_agent="localizacao")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df_ceps_clientes['BAIRRO_CIDADE_GEOLOCALIZADO'] = df_ceps_clientes['BAIRRO_CIDADE'].apply(geocode)
df_ceps_clientes.head()

,CEP_CLIENTE,LOGRADOURO_CLIENTE,BAIRRO_CLIENTE,CIDADE_CLIENTE,ESTADO_CLIENTE,BAIRRO_CIDADE,BAIRRO_CIDADE_GEOLOCALIZADO
0,78795000,,,Pedra Preta,MT,-Pedra Preta,"(Pedra Preta, Região Geográfica Imediata de Ro..."
1,74905090,Rua Santa Helena,Jardim das Esmeraldas,Aparecida de Goiânia,GO,Jardim das Esmeraldas-Aparecida de Goiânia,None
2,78725160,Rua Elza Oliveira,Jardim Primavera II,Rondonópolis,MT,Jardim Primavera II-Rondonópolis,"(Jardim Primavera, Rondonópolis, Região Geográ..."
3,75904520,Rua 7,Setor Dona Gercina,Rio Verde,GO,Setor Dona Gercina-Rio Verde,None
4,43900000,,,São Francisco do Conde,BA,-São Francisco do Conde,"(São Francisco do Conde, Região Geográfica Ime..."


In [19]:
df_ceps_vendedor['BAIRRO_CIDADE_GEOLOCALIZADO'].isna().sum()

1

In [20]:
df_ceps_vendedor.dropna(inplace=True)

In [21]:
df_ceps_vendedor

,CEP_VENDEDOR,LOGRADOURO_VENDEDOR,BAIRRO_VENDEDOR,CIDADE_VENDEDOR,ESTADO_VENDEDOR,BAIRRO_CIDADE,BAIRRO_CIDADE_GEOLOCALIZADO
0,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,Nova Suiça-Goiânia,"(Setor Nova Suiça, Goiânia, Região Geográfica ..."
3,74535290,Rua Doutor Gil Lino,Setor Coimbra,Goiânia,GO,Setor Coimbra-Goiânia,"(Setor Coimbra, Goiânia, Região Geográfica Ime..."
4,40385640,Rua José Tibério,Boa Vista de São Caetano,Salvador,BA,Boa Vista de São Caetano-Salvador,"(Boa Vista de São Caetano, Salvador, Região Ge..."
5,14080210,Rua Capitão Salomão,Campos Elíseos,Ribeirão Preto,SP,Campos Elíseos-Ribeirão Preto,"(Campos Elíseos, Ribeirão Preto, Região Imedia..."
6,74883020,Rua Iraí,Alphaville Flamboyant Residencial Araguaia,Goiânia,GO,Alphaville Flamboyant Residencial Araguaia-Goi...,"(Alphaville Flamboyant Residencial Araguaia, G..."
9,38401515,Rua Quatorze de Julho,Pacaembu,Uberlândia,MG,Pacaembu-Uberlândia,"(Pacaembú, Setor Norte, Uberlândia, Região Geo..."
10,65070020,Rua Alfa Centauri,Recanto dos Vinhais,São Luís,MA,Recanto dos Vinhais-São Luís,"(Recanto dos Vinhais, São Luís, Região Geográf..."
12,07097360,Rua Mariza,Jardim Flor da Montanha,Guarulhos,SP,Jardim Flor da Montanha-Guarulhos,(Vila Jardim Flor da Montanha - Jardim Santa M...
51,38408482,Rua São Francisco de Assis,Lagoinha,Uberlândia,MG,Lagoinha-Uberlândia,"(Lagoinha, Setor Sul, Uberlândia, Região Geogr..."
75,75800012,Avenida Goiás,Centro,Jataí,GO,Centro-Jataí,"(Jataí, Região Geográfica Imediata de Jataí-Mi..."


In [22]:
df_ceps_vendedor['LATITUDE_VENDEDOR'] = df_ceps_vendedor['BAIRRO_CIDADE_GEOLOCALIZADO'].apply(lambda loc: loc.latitude)
df_ceps_vendedor['LONGITUDE_VENDEDOR'] = df_ceps_vendedor['BAIRRO_CIDADE_GEOLOCALIZADO'].apply(lambda loc: loc.longitude)
df_ceps_vendedor.drop('BAIRRO_CIDADE', axis=1, inplace=True)
df_ceps_vendedor.drop('BAIRRO_CIDADE_GEOLOCALIZADO', axis=1, inplace=True)
df_ceps_vendedor

,CEP_VENDEDOR,LOGRADOURO_VENDEDOR,BAIRRO_VENDEDOR,CIDADE_VENDEDOR,ESTADO_VENDEDOR,LATITUDE_VENDEDOR,LONGITUDE_VENDEDOR
0,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,-16.714054,-49.273101
3,74535290,Rua Doutor Gil Lino,Setor Coimbra,Goiânia,GO,-16.677482,-49.291481
4,40385640,Rua José Tibério,Boa Vista de São Caetano,Salvador,BA,-12.925900,-38.476500
5,14080210,Rua Capitão Salomão,Campos Elíseos,Ribeirão Preto,SP,-21.157718,-47.798553
6,74883020,Rua Iraí,Alphaville Flamboyant Residencial Araguaia,Goiânia,GO,-16.700305,-49.214774
9,38401515,Rua Quatorze de Julho,Pacaembu,Uberlândia,MG,-18.884708,-48.291262
10,65070020,Rua Alfa Centauri,Recanto dos Vinhais,São Luís,MA,-2.521034,-44.264078
12,07097360,Rua Mariza,Jardim Flor da Montanha,Guarulhos,SP,-23.448414,-46.544632
51,38408482,Rua São Francisco de Assis,Lagoinha,Uberlândia,MG,-18.929949,-48.258115
75,75800012,Avenida Goiás,Centro,Jataí,GO,-17.878383,-51.720424


In [23]:
df_ceps_clientes.dropna(inplace=True)

In [24]:
df_ceps_clientes['LATITUDE_CLIENTE'] = df_ceps_clientes['BAIRRO_CIDADE_GEOLOCALIZADO'].apply(lambda loc: loc.latitude)
df_ceps_clientes['LONGITUDE_CLIENTE'] = df_ceps_clientes['BAIRRO_CIDADE_GEOLOCALIZADO'].apply(lambda loc: loc.longitude)
df_ceps_clientes.drop('BAIRRO_CIDADE', axis=1, inplace=True)
df_ceps_clientes.drop('BAIRRO_CIDADE_GEOLOCALIZADO', axis=1, inplace=True)
df_ceps_clientes

,CEP_CLIENTE,LOGRADOURO_CLIENTE,BAIRRO_CLIENTE,CIDADE_CLIENTE,ESTADO_CLIENTE,LATITUDE_CLIENTE,LONGITUDE_CLIENTE
0,78795000,,,Pedra Preta,MT,-16.623002,-54.476123
2,78725160,Rua Elza Oliveira,Jardim Primavera II,Rondonópolis,MT,-16.449492,-54.614710
4,43900000,,,São Francisco do Conde,BA,-12.628108,-38.681133
5,14061350,Rua Jovino Campos,Geraldo Correia de Carvalho,Ribeirão Preto,SP,-21.151088,-47.817075
7,74491628,Rua JC311,Residencial Jardins do Cerrado 7,Goiânia,GO,-16.674007,-49.438824
8,75392702,Rua 6,Vila Maria,Trindade,GO,-27.787940,-50.322328
9,38410214,Rua Norival Pereira Alves,São Jorge,Uberlândia,MG,-18.960517,-48.224862
10,65110000,,,São José de Ribamar,MA,-2.560912,-44.055951
11,74711010,Alameda do Palmito,Vila Bandeirantes,Goiânia,GO,-16.671640,-49.225780
12,72410305,Quadra 5 Conjunto E,Setor Sul (Gama),Brasília,DF,-16.028436,-48.073314


In [25]:
df_tratado = pd.merge(df_original, df_ceps_vendedor, how='inner', on = 'CEP_VENDEDOR')
df_tratado = pd.merge(df_tratado, df_ceps_clientes, how='inner', on = 'CEP_CLIENTE')
df_tratado.head()

,CODIGO_CLIENTE,DATA_VENDA,CODIGO_VENDEDOR,CEP_CLIENTE,CEP_VENDEDOR,LOGRADOURO_VENDEDOR,BAIRRO_VENDEDOR,CIDADE_VENDEDOR,ESTADO_VENDEDOR,LATITUDE_VENDEDOR,LONGITUDE_VENDEDOR,LOGRADOURO_CLIENTE,BAIRRO_CLIENTE,CIDADE_CLIENTE,ESTADO_CLIENTE,LATITUDE_CLIENTE,LONGITUDE_CLIENTE
0,5695,11/09/2015,969022,78795000,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,-16.714054,-49.273101,,,Pedra Preta,MT,-16.623002,-54.476123
1,5711,22/09/2015,969022,78725160,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,-16.714054,-49.273101,Rua Elza Oliveira,Jardim Primavera II,Rondonópolis,MT,-16.449492,-54.614710
2,6288,25/05/2016,969022,74711010,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,-16.714054,-49.273101,Alameda do Palmito,Vila Bandeirantes,Goiânia,GO,-16.671640,-49.225780
3,7068,28/09/2016,969022,74130011,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,-16.714054,-49.273101,Avenida Assis Chateaubriand,Setor Oeste,Goiânia,GO,-16.681220,-49.272167
4,7625,24/02/2017,969022,75383585,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,-16.714054,-49.273101,Rua Joaquim Mathias,Setor Cristina,Trindade,GO,-16.635246,-49.418521


In [26]:
#precisa passar as duas distancias do cliente e vendedor (lat e lon)

for idx, row in df_tratado.iterrows():
    
    coord_vendedor = (row.LATITUDE_VENDEDOR, row.LONGITUDE_VENDEDOR)
    coord_cliente = (row.LATITUDE_CLIENTE, row.LONGITUDE_CLIENTE)    
    
    df_tratado.at[idx, 'DISTANCIA_KM_VENDEDOR_CLIENTE'] = round(geopy.distance.distance(coord_vendedor, coord_cliente).km,2)  

In [27]:
df_tratado.head(10)

,CODIGO_CLIENTE,DATA_VENDA,CODIGO_VENDEDOR,CEP_CLIENTE,CEP_VENDEDOR,LOGRADOURO_VENDEDOR,BAIRRO_VENDEDOR,CIDADE_VENDEDOR,ESTADO_VENDEDOR,LATITUDE_VENDEDOR,LONGITUDE_VENDEDOR,LOGRADOURO_CLIENTE,BAIRRO_CLIENTE,CIDADE_CLIENTE,ESTADO_CLIENTE,LATITUDE_CLIENTE,LONGITUDE_CLIENTE,DISTANCIA_KM_VENDEDOR_CLIENTE
0,5695,11/09/2015,969022,78795000,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,-16.714054,-49.273101,,,Pedra Preta,MT,-16.623002,-54.476123,555.09
1,5711,22/09/2015,969022,78725160,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,-16.714054,-49.273101,Rua Elza Oliveira,Jardim Primavera II,Rondonópolis,MT,-16.449492,-54.614710,570.79
2,6288,25/05/2016,969022,74711010,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,-16.714054,-49.273101,Alameda do Palmito,Vila Bandeirantes,Goiânia,GO,-16.671640,-49.225780,6.89
3,7068,28/09/2016,969022,74130011,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,-16.714054,-49.273101,Avenida Assis Chateaubriand,Setor Oeste,Goiânia,GO,-16.681220,-49.272167,3.63
4,7625,24/02/2017,969022,75383585,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,-16.714054,-49.273101,Rua Joaquim Mathias,Setor Cristina,Trindade,GO,-16.635246,-49.418521,17.80
5,7740,22/06/2017,969022,74810240,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,-16.714054,-49.273101,Rua 56,Jardim Goiás,Goiânia,GO,-16.700797,-49.237211,4.10
6,7751,22/06/2017,969022,74810240,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,-16.714054,-49.273101,Rua 56,Jardim Goiás,Goiânia,GO,-16.700797,-49.237211,4.10
7,1395276,08/10/2015,969022,22250040,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,-16.714054,-49.273101,Praia Botafogo,Botafogo,Rio de Janeiro,RJ,-22.948832,-43.180122,939.79
8,1395281,14/10/2015,969022,78735775,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,-16.714054,-49.273101,Rua Noruega,Jardim Europa,Rondonópolis,MT,-16.470578,-54.573490,566.25
9,1395300,06/10/2015,969022,78730231,74280380,Avenida T 15,Nova Suiça,Goiânia,GO,-16.714054,-49.273101,Rua Rosalino Antônio Silva,Núcleo Habitacional Participação,Rondonópolis,MT,-16.464329,-54.608459,570.02
